In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

%matplotlib inline
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from neuralforecast.auto import AutoTFT
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.core import NeuralForecast

from datetime import datetime, timedelta
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from neuralforecast.losses.pytorch import MQLoss
from tqdm import tqdm

In [2]:
# Load and process the MEASLES_ARIZONA data
df = pd.read_csv("outbreaks_disease_location.csv")
value_columns = [str(i) for i in range(60)]
series_values = df[value_columns].fillna(0).astype(float)
start_dates = pd.to_datetime(df["start_date"])

# Shuffle and split
shuffled_indices = df.sample(frac=1, random_state=42).index
split_point = int(0.8 * len(df))
train_indices = shuffled_indices[:split_point]
test_indices = shuffled_indices[split_point:]

In [3]:
train_records = []
for i, row in series_values.iloc[train_indices].iterrows():
    dates = pd.date_range(start="2000-01-01", periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        train_records.append({"unique_id": f"Y{i+1}", "ds": dates[t], "y": value})
df_train = pd.DataFrame(train_records)

test_start_dates = start_dates.loc[test_indices] - pd.Timedelta(weeks=4)
df_test_all = []

for idx in test_indices:
    start_date = test_start_dates.loc[idx]
    row = series_values.loc[idx]
    dates = pd.date_range(start=start_date, periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        df_test_all.append({"unique_id": f"Y_{idx}", "ds": dates[t], "y": value})

df_test_all = pd.DataFrame(df_test_all)

In [4]:
df_train["date"] = pd.to_datetime(df_train["ds"])
df_train.set_index("date", inplace = True)

df_test_all["date"] = pd.to_datetime(df_test_all["ds"])
df_test_all.set_index("date", inplace = True)

In [5]:
df_test_all

,unique_id,ds,y
date,,,
2022-08-13,Y_5607,2022-08-13,20775.0
2022-08-20,Y_5607,2022-08-20,16570.0
2022-08-27,Y_5607,2022-08-27,13995.0
2022-09-03,Y_5607,2022-09-03,12145.0
2022-09-10,Y_5607,2022-09-10,10686.0
...,...,...,...
1939-05-20,Y_7270,1939-05-20,0.0
1939-05-27,Y_7270,1939-05-27,0.0
1939-06-03,Y_7270,1939-06-03,0.0


In [6]:
class FixedModelTFTProcessor:
    def __init__(self, dates=[]):
        self.dates  = dates
        self.forecasts, self.eval_pairs = [], []
        self.testset = None
        self.nf, self.model = None, None
        self.selected_input_size = None
        self.maes, self.mses, self.mapes, self.nmses = [], [], [], []
        self.metrics_df  = pd.DataFrame(columns=["Unique_id","Reference Date","MAE","MSE","MAPE","NMSE"])
        self.display_df  = pd.DataFrame(columns=["Unique_id","Reference Date","Target End Date","GT","Quantile","Prediction"])
        self.wis_df      = pd.DataFrame()

    # ──────────────────────────────────────────
    def create_fixed_model(self, df_train, df_test, h: int, freq: str, level=[]):
        min_len = df_train.groupby("unique_id").size().min()
        max_in  = max(8, min_len - h - 1)
        self.testset = df_test

        def cfg(t):
            return {
                "input_size"   : t.suggest_int("input_size", 8, max_in),
                "learning_rate": t.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
                "batch_size"   : t.suggest_categorical("batch_size",[16,32,64]),
                "random_seed"  : t.suggest_int("random_seed",1,99999),
                "accelerator":"gpu","devices":2,"strategy":"auto"
            }

        model = AutoTFT(
            h=h, backend="optuna",
            loss=MQLoss(level=level) if level else None,
            config=cfg)

        self.nf = NeuralForecast(models=[model], freq=freq)
        self.nf.fit(df=df_train)
        self.model = self.nf.models[0].model
        self.selected_input_size = self.model.hparams["input_size"]

    # ──────────────────────────────────────────
    def sliding_batch_by_index(self, h: int = 4):
        """Return merged forecast+truth rows for every UID / reference date."""
        uid_dates = {u: g["ds"].sort_values().to_numpy()
                     for u, g in self.testset.groupby("unique_id")}
        T, s = min(len(d) for d in uid_dates.values()), self.selected_input_size
        rows = []

        for t in tqdm(range(s, T - h + 1), desc="index t"):
            batch, refs = [], []
            for uid, dates in uid_dates.items():
                win, ref = dates[t-s:t], dates[t-1]
                df_u  = self.testset[self.testset["unique_id"] == uid]
                df_in = df_u[df_u["ds"].isin(win)].copy()
                if len(df_in) == s:
                    df_in["unique_id"] = uid
                    batch.append(df_in); refs.append((uid, ref))
            if not batch: continue

            fcst_batch = self.nf.predict(df=pd.concat(batch)).set_index("ds")
            for uid, ref in refs:
                f  = fcst_batch[fcst_batch["unique_id"] == uid].reset_index()
                gt = (self.testset[self.testset["unique_id"] == uid]
                      .set_index("ds").loc[f["ds"]].reset_index())
                merged = pd.merge(f, gt, on=["unique_id","ds"], how="inner")
                merged["Reference Date"] = pd.to_datetime(ref).strftime("%Y-%m-%d")
                rows.append(merged)

        return pd.concat(rows, ignore_index=True)

    # ──────────────────────────────────────────
    def create_display_df(self, merged_df, model_name="AutoTFT"):
        recs = []
        for _, g in tqdm(merged_df.groupby("unique_id"), desc="display"):
            uid = g["unique_id"].iloc[0]; ref = g["Reference Date"].iloc[0]
            for col in g.columns:
                if not col.startswith(model_name): continue
                if col.endswith("median") or col == model_name:
                    q = 0.5
                elif "-lo-" in col:
                    lvl = int(col.split("-")[-1]); q = (1-lvl/100)/2
                elif "-hi-" in col:
                    lvl = int(col.split("-")[-1]); q = 1-(1-lvl/100)/2
                else:
                    continue
                tmp = g[["ds", col, "y"]].rename(columns={"ds":"Target End Date",
                                                           col:"Prediction",
                                                           "y":"GT"})
                tmp["Unique_id"] = uid
                tmp["Reference Date"] = ref
                tmp["Quantile"] = q
                recs.append(tmp)
        self.display_df = (pd.concat(recs)
                           .loc[:,["Unique_id","Reference Date","Target End Date",
                                   "GT","Quantile","Prediction"]]
                           .sort_values(["Unique_id","Reference Date",
                                         "Target End Date","Quantile"])
                           .reset_index(drop=True))

    # ──────────────────────────────────────────
    def efficient_compute_wis(self):
        if self.display_df.empty:
            raise ValueError("Run create_display_df first.")
        rows, df = [], self.display_df.sort_values(
            ["Unique_id","Reference Date","Target End Date","Quantile"])
        for (uid, ref, tgt), grp in tqdm(df.groupby(
                ["Unique_id","Reference Date","Target End Date"]), desc="WIS"):
            gt = grp["GT"].iloc[0]
            preds = grp.set_index("Quantile")["Prediction"]
            if 0.5 not in preds.index: continue
            rows.append({"Unique_id":uid,"Reference Date":ref,
                         "Target End Date":tgt,"GT":gt,
                         "WIS": abs(preds[0.5]-gt)})   # AE-only shortcut
        self.wis_df = pd.DataFrame(rows)

    def calculate_metrics(self):
        for fc, gt in self.eval_pairs:
            y_true, y_pred = gt.iloc[:,1], fc.iloc[:,0]
            self.maes.append(mean_absolute_error(y_true, y_pred))
            self.mses.append(mean_squared_error(y_true, y_pred))
            self.mapes.append(mean_absolute_percentage_error(y_true, y_pred))
            self.nmses.append(self.mses[-1]/np.var(y_true) if np.var(y_true)!=0 else np.nan)

    def create_metrics_df(self):
        for i, (fc, _) in enumerate(self.eval_pairs):
            uid = fc["unique_id"].iloc[0]
            self.metrics_df.loc[len(self.metrics_df)] = [
                uid, self.dates[i], self.maes[i],
                self.mses[i], self.mapes[i], self.nmses[i]
            ]

In [7]:
test_ids = [f"Y_{i}" for i in test_indices]

In [8]:
processor = FixedModelTFTProcessor()

In [9]:
processor.create_fixed_model(df_train, df_test_all, h=4, freq="W-SAT", level = [10,20,30,40,50,60,70,80, 85,90,95])

[I 2025-06-25 19:43:26,624] A new study created in memory with name: no-name-d10b6e0b-671b-4528-b928-bcd9b2fdaa0d
[rank: 0] Seed set to 94920
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | MQLoss                   | 23     | train
1 | padder_train            | C

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.


In [10]:
results = []

In [11]:
processor.selected_input_size

48

In [12]:
all_results = processor.sliding_batch_by_index(h=4)

index t:   0%|          | 0/9 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:  11%|█         | 1/9 [00:34<04:35, 34.41s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:  22%|██▏       | 2/9 [01:07<03:57, 33.88s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

index t:  78%|███████▊  | 7/9 [03:57<01:07, 33.83s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:  89%|████████▉ | 8/9 [04:31<00:33, 33.99s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t: 100%|██████████| 9/9 [05:05<00:00, 33.92s/it]


In [13]:
all_results.to_csv('TFT_concatenated.csv')

In [14]:
all_results

,ds,unique_id,AutoTFT-median,AutoTFT-lo-95,AutoTFT-lo-90,AutoTFT-lo-85,AutoTFT-lo-80,AutoTFT-lo-70,AutoTFT-lo-60,AutoTFT-lo-50,...,AutoTFT-hi-40,AutoTFT-hi-50,AutoTFT-hi-60,AutoTFT-hi-70,AutoTFT-hi-80,AutoTFT-hi-85,AutoTFT-hi-90,AutoTFT-hi-95,y,Reference Date
0,2011-08-20,Y_1,0.005639,-0.129357,-0.014623,-0.009914,-0.003494,-0.007165,0.002337,0.004187,...,0.012961,0.005720,0.014237,0.023014,0.032376,0.032619,0.027741,0.020599,0.0,2011-08-13
1,2011-08-27,Y_1,0.007544,-0.164460,-0.034992,-0.013188,-0.004945,-0.006904,-0.002240,0.007320,...,0.013444,0.007461,0.017399,0.025384,0.028290,0.032263,0.024909,0.028731,0.0,2011-08-13
2,2011-09-03,Y_1,0.006718,-0.185243,-0.044292,-0.015015,-0.005589,-0.009233,-0.000465,0.007453,...,0.014516,0.005732,0.017846,0.023626,0.027102,0.030087,0.021266,0.028455,0.0,2011-08-13
3,2011-09-10,Y_1,0.005767,-0.193566,-0.049198,-0.014359,-0.006170,-0.009594,0.001008,0.006760,...,0.014015,0.004583,0.016259,0.023023,0.026448,0.029563,0.019160,0.027068,0.0,2011-08-13
4,2022-04-16,Y_1000,1.735596,-116.181396,-13.031494,-10.886902,-5.684509,-8.377197,-0.371704,0.465881,...,6.858154,0.953186,7.634216,13.297241,22.224548,21.882812,20.512268,15.642029,0.0,2022-04-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77755,1955-04-16,Y_9998,0.016347,-1.216614,-0.414596,-0.329703,-0.167789,-0.180971,-0.121724,-0.050518,...,0.075242,0.093885,0.114374,0.140877,0.199515,0.255007,0.325054,0.959210,0.0,1955-03-19
77756,1957-03-02,Y_9999,0.574305,-0.929269,-0.819821,-0.971050,-0.519633,-0.432413,-0.077535,0.065995,...,0.984272,1.124398,1.253087,1.294416,1.633808,2.103850,1.809075,2.365060,0.0,1957-02-23
77757,1957-03-09,Y_9999,0.527419,-1.102303,-1.005127,-1.093238,-0.681084,-0.544092,-0.199233,-0.038151,...,0.912689,1.080675,1.238199,1.259047,1.587734,2.085230,1.793015,2.379684,0.0,1957-02-23
77758,1957-03-16,Y_9999,0.494719,-1.205756,-1.123142,-1.188925,-0.786705,-0.620863,-0.278912,-0.107438,...,0.876445,1.052427,1.232166,1.237848,1.563504,2.064207,1.770174,2.378098,0.0,1957-02-23


In [15]:
df = all_results.rename(columns={
    "ds"        : "Target End Date",
    "unique_id" : "Unique_id",
    "y"         : "GT"
})

# long‐format all quantile columns (everything that starts with AutoNBEATS-)
quantile_cols = [c for c in df.columns
                 if c.startswith("AutoTFT-")]

df_long = df.melt(
    id_vars = ["Unique_id", "Reference Date", "Target End Date", "GT"],
    value_vars = quantile_cols,
    var_name = "QuantileLabel",
    value_name = "Prediction"
)

def to_quantile(label: str) -> float:
    if "median" in label: return 0.5
    if "-lo-" in label:
        lvl = int(label.split("-")[-1]); return (1 - lvl/100) / 2
    if "-hi-" in label:
        lvl = int(label.split("-")[-1]); return 1 - (1 - lvl/100) / 2
    raise ValueError(label)

df_long["Quantile"] = df_long["QuantileLabel"].apply(to_quantile)

rows = []

group_iter = df_long.groupby(["Unique_id", "Reference Date", "Target End Date"])
for (uid, ref_date, tgt_date), g in tqdm(group_iter,
                                         total=len(group_iter),
                                         desc="point-wise metrics"):

    gt = g["GT"].iloc[0]
    preds = g.set_index("Quantile")["Prediction"]

    if 0.5 not in preds:          # need the median
        continue

    # point errors
    y_pred = preds[0.5]
    mae  = abs(y_pred - gt)
    mse  = (y_pred - gt) ** 2
    mape = abs((y_pred - gt) / gt) if gt else np.nan
    nmse = np.nan                # single point ⇒ var = 0

    # WIS for this timestamp
    qs  = sorted(q for q in preds.index if q != 0.5)
    n   = len(qs) // 2
    ae  = mae
    int_scores = 0
    for i in range(n):
        lo_q, hi_q = qs[i], qs[-(i + 1)]
        lo, hi     = preds[lo_q], preds[hi_q]
        alpha      = hi_q - lo_q
        int_scores += (hi - lo) \
                    + (2/alpha) * max(lo - gt, 0) \
                    + (2/alpha) * max(gt - hi, 0)

    wis = (ae + int_scores) / (1 + n)

    rows.append({
        "Unique_id"      : uid,
        "Reference Date" : ref_date,
        "Target End Date": tgt_date,
        "GT"             : gt,
        "Prediction"     : y_pred,
        "MAE"            : mae,
        "MSE"            : mse,
        "MAPE"           : mape,
        "NMSE"           : nmse,
        "WIS"            : wis
    })

pointwise_df = pd.DataFrame(rows).sort_values(
    ["Unique_id", "Reference Date", "Target End Date"])

point-wise metrics: 100%|██████████| 77760/77760 [00:37<00:00, 2053.88it/s]


In [16]:
pointwise_df

,Unique_id,Reference Date,Target End Date,GT,Prediction,MAE,MSE,MAPE,NMSE,WIS
0,Y_1,2011-08-13,2011-08-20,0.0,0.005639,0.005639,0.000032,NaN,NaN,0.052901
1,Y_1,2011-08-13,2011-08-27,0.0,0.007544,0.007544,0.000057,NaN,NaN,0.062390
2,Y_1,2011-08-13,2011-09-03,0.0,0.006718,0.006718,0.000045,NaN,NaN,0.061988
3,Y_1,2011-08-13,2011-09-10,0.0,0.005767,0.005767,0.000033,NaN,NaN,0.063309
4,Y_1,2011-08-20,2011-08-27,0.0,0.004501,0.004501,0.000020,NaN,NaN,0.043252
...,...,...,...,...,...,...,...,...,...,...
77755,Y_9999,1957-02-16,1957-03-16,0.0,0.951957,0.951957,0.906222,NaN,NaN,4.774657
77756,Y_9999,1957-02-23,1957-03-02,0.0,0.574305,0.574305,0.329826,NaN,NaN,3.077226
77757,Y_9999,1957-02-23,1957-03-09,0.0,0.527419,0.527419,0.278171,NaN,NaN,2.829010
77758,Y_9999,1957-02-23,1957-03-16,0.0,0.494719,0.494719,0.244746,NaN,NaN,2.674047


In [17]:
pointwise_df.to_csv('TFT_ALL_STATS.csv')